# SSA Object for IVOA protocols (v1.2)
    v1.2 -- Updated to Spectral Data Model v2.0

In [1]:
# imports
from astropy.io.votable.tree import VOTableFile, Resource, Field, Info, Param
from astropy.io.votable.tree import Table as voTable
from astropy.io.votable import from_table

from astropy.coordinates import SkyCoord

import specdb
from specdb import specdb as ssdb
from specdb import ssa as spdb_ssa

## Fiddling with QueryData and VOTables

### Load

In [2]:
tst_file = specdb.__path__[0]+'/tests/files/IGMspec_DB_v02_debug.hdf5'

In [3]:
reload(ssdb)
igmsp = ssdb.IgmSpec(db_file=tst_file, verbose=True)

Using /Users/xavier/local/Python/specdb/specdb/tests/files/IGMspec_DB_v02_debug.hdf5 for the DB file
Available groups: [u'BOSS_DR12', u'HD-LLS_DR1', u'GGG', u'SDSS_DR7']
Database is igmspec
Created on 2017-Jan-07


### Generate VOTable

In [8]:
reload(spdb_ssa)
ssai = spdb_ssa.SSAInterface(igmsp)

In [9]:
votable = ssai.querydata('0.0019,17.7737', SIZE=1e-3)

Your search yielded 1 match[es] within radius=0.001 deg


In [10]:
votable.to_xml('tmp.xml')

### Here is what it looks like
    <?xml version="1.0" encoding="utf-8"?>
    <!-- Produced with astropy.io.votable version 1.3
         http://www.astropy.org/ -->
    <VOTABLE version="1.2" xmlns="http://www.ivoa.net/xml/VOTable/v1.2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.ivoa.net/xml/VOTable/v1.2">
     <RESOURCE type="results">
      <INFO ID="QUERY_STATUS" name="QUERY_STATUS" value="OK"/>
      <INFO ID="SERVICE_PROTOCOL" name="SERVICE_PROTOCOL" value="1.1"/>
      <INFO ID="REQUEST" name="REQUEST" value="queryData"/>
      <INFO ID="serviceName" name="serviceName" value="ssap"/>
      <INFO ID="FORMAT" name="FORMAT" value="HDF5"/>
      <INFO ID="SIZE" name="SIZE" value="0.001"/>
      <INFO ID="POS" name="POS" value="0.0019,17.7737"/>
      <TABLE>
       <FIELD ID="DataModel" arraysize="12" datatype="char" name="DataModel" utype="ssa:Dataset.DataModel"/>
       <FIELD ID="DatasetType" arraysize="8" datatype="char" name="DatasetType" utype="ssa:Dataset.Type"/>
       <FIELD ID="Title" arraysize="28" datatype="char" name="Title" ucd="meta.title;meta.dataset" utype="ssa:DataID.Title"/>
       <FIELD ID="Instrument" arraysize="4" datatype="char" name="Instrument" ucd="meta.id;instr" utype="ssa:DataID.Title"/>
       <FIELD ID="Publisher" arraysize="6" datatype="char" name="Publisher" ucd=" meta.curation" utype="ssa:Curation.Publisher"/>
       <FIELD ID="SpaceFrameName" arraysize="4" datatype="char" name="SpaceFrameName" utype="ssa:CoordSys.SpaceFrame.Name"/>
       <FIELD ID="SpaceFrameEquinox" datatype="double" name="SpaceFrameEquinox" ucd="time.equinox;pos.frame" unit="yr" utype="ssa:CoordSys.SpaceFrame.Equinox"/>
       <FIELD ID="TargetName" arraysize="11" datatype="char" name="TargetName" ucd="meta.id;src" utype="ssa:Target.Name"/>
       <FIELD ID="FluxAxisUcd" arraysize="18" datatype="char" name="FluxAxisUcd" utype="ssa:Char.FluxAxis.ucd"/>
       <FIELD ID="FluxAxisUnit" arraysize="26" datatype="char" name="FluxAxisUnit" utype="ssa:Char.FluxAxis.unit"/>
       <FIELD ID="FluxAxisCalibration" arraysize="8" datatype="char" name="FluxAxisCalibration" utype="ssa:Char.FluxAxis.Calibration"/>
       <FIELD ID="SpectralAxisUcd" arraysize="5" datatype="char" name="SpectralAxisUcd" utype="ssa:Char.SpectralAxis.ucd"/>
       <FIELD ID="SpectralAxisUnit" arraysize="8" datatype="char" name="SpectralAxisUnit" utype="ssa:Char.SpectralAxis.unit"/>
       <FIELD ID="TimeLocation" datatype="double" name="TimeLocation" ucd="time.epoch" unit="d" utype="ssa:Char.TimeAxis.Coverage.Location.Value"/>
       <FIELD ID="SpatialLocation" arraysize="2" datatype="double" name="SpatialLocation" ucd="pos.eq" unit="deg" utype="ssa:Char.SpatialAxis.Coverage.Location.Value"/>
       <FIELD ID="SpectralLocation" datatype="double" name="SpectralLocation" ucd="instr.bandpass" unit="AA" utype="ssa:Char.Spectral.Coverage.Location.Value"/>
       <FIELD ID="SpectralBoundsExtent" datatype="double" name="SpectralBoundsExtent" ucd="instr.bandwidth" unit="AA" utype="ssa:Char.Spectral.Coverage.Bounds.Extent"/>
       <FIELD ID="SpectralBoundsStart" datatype="double" name="SpectralBoundsStart" ucd="em.wl;stat.min" unit="AA" utype="ssa:Char.Spectral.Coverage.Bounds.Start"/>
       <FIELD ID="SpectralBoundsStop" datatype="double" name="SpectralBoundsStop" ucd="em.wl;stat.max" unit="AA" utype="ssa:Char.Spectral.Coverage.Bounds.Stop"/>
       <DATA>
        <TABLEDATA>
         <TR>
          <TD>Spectrum-2.0</TD>
          <TD>Spectrum</TD>
          <TD>BOSS_DR12: BOSS DR12 Quasars</TD>
          <TD>BOSS</TD>
          <TD>specdb</TD>
          <TD>ICRS</TD>
          <TD>2000</TD>
          <TD>BOSS_DR12_0</TD>
          <TD>phot.fluDens;em.wl</TD>
          <TD>erg s**(-1) angstrom**(-1)</TD>
          <TD>ABSOLUTE</TD>
          <TD>em.wl</TD>
          <TD>Angstrom</TD>
          <TD>56238</TD>
          <TD>0.001898285183756343 17.773739129913793</TD>
          <TD>6947.298095703125</TD>
          <TD>6684.68310546875</TD>
          <TD>3604.95654296875</TD>
          <TD>10289.6396484375</TD>
         </TR>
        </TABLEDATA>
       </DATA>
      </TABLE>
     </RESOURCE>
    </VOTABLE>


## METADATA Query

In [11]:
reload(spdb_ssa)
ssai = spdb_ssa.SSAInterface(igmsp)

In [12]:
votable = ssai.querydata(FORMAT='METADATA')

In [13]:
votable.to_xml('meta.xml')

### Here is the output:
    <?xml version="1.0" encoding="utf-8"?>
    <!-- Produced with astropy.io.votable version 1.3
         http://www.astropy.org/ -->
    <VOTABLE version="1.2" xmlns="http://www.ivoa.net/xml/VOTable/v1.2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.ivoa.net/xml/VOTable/v1.2">
     <RESOURCE type="results">
      <PARAM ID="INPUT_POS" arraysize="*" datatype="char" name="INPUT:POS" value="">
       <DESCRIPTION>
        The center of the region of interest. The coordinate values are
        specified in list format (comma separated) in decimal degrees with
        no embedded white space followed by an optional coord. system.
        Allowed systems are (ICRS) and the default is ICRS.
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="INPUT_SIZE" datatype="double" name="INPUT:SIZE" unit="deg" value="0.1">
       <DESCRIPTION>
        The radius of the circular region of interest in decimal
        degrees.Default sized radius is 0.001 degrees
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="INPUT_BAND" arraysize="*" datatype="char" name="INPUT:BAND" value="ALL">
       <DESCRIPTION>
        Not currently implemented
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="INPUT_TIME" arraysize="*" datatype="char" name="INPUT:TIME" value="">
       <DESCRIPTION>
        Not currently implemented
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="INPUT_FORMAT" arraysize="*" datatype="char" name="INPUT:FORMAT" value="ALL">
       <DESCRIPTION>
        Desired format of retrieved data.  Allowed values are HDF5,
        METADATA
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="DataModel" arraysize="*" datatype="char" name="OUTPUT:DataModel" utype="ssa:Dataset.DataModel" value="">
       <DESCRIPTION>
        Datamodel name and version
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="DatasetType" arraysize="*" datatype="char" name="OUTPUT:DatasetType" utype="ssa:Dataset.Type" value="Spectrum">
       <DESCRIPTION>
        Dataset type
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="Title" arraysize="*" datatype="char" name="OUTPUT:Title" ucd="meta.title;meta.dataset" utype="ssa:DataID.Title" value="">
       <DESCRIPTION>
        Dataset Title
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="Instrument" arraysize="*" datatype="char" name="OUTPUT:Instrument" ucd="meta.id;instr" utype="ssa:DataID.Title" value="">
       <DESCRIPTION>
        Instrument name
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="Publisher" arraysize="*" datatype="char" name="OUTPUT:Publisher" ucd=" meta.curation" utype="ssa:Curation.Publisher" value=""/>
      <PARAM ID="SpaceFrameName" arraysize="*" datatype="char" name="OUTPUT:SpaceFrameName" utype="ssa:CoordSys.SpaceFrame.Name" value="">
       <DESCRIPTION>
        Spatial coordinate frame name
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="SpaceFrameEquinox" datatype="double" name="OUTPUT:SpaceFrameEquinox" ucd="time.equinox;pos.frame" unit="yr" utype="ssa:CoordSys.SpaceFrame.Equinox" value="0"/>
      <PARAM ID="TargetName" arraysize="1" datatype="char" name="OUTPUT:TargetName" ucd="meta.id;src" utype="ssa:Target.Name" value=""/>
      <PARAM ID="FluxAxisUcd" arraysize="*" datatype="char" name="OUTPUT:FluxAxisUcd" utype="ssa:Char.FluxAxis.ucd" value=""/>
      <PARAM ID="FluxAxisUnit" arraysize="*" datatype="char" name="OUTPUT:FluxAxisUnit" utype="ssa:Char.FluxAxis.unit" value=""/>
      <PARAM ID="FluxAxisCalibration" arraysize="*" datatype="char" name="OUTPUT:FluxAxisCalibration" utype="ssa:Char.FluxAxis.Calibration" value=""/>
      <PARAM ID="SpectralAxisUcd" arraysize="*" datatype="char" name="OUTPUT:SpectralAxisUcd" utype="ssa:Char.SpectralAxis.ucd" value=""/>
      <PARAM ID="SpectralAxisUnit" arraysize="*" datatype="char" name="OUTPUT:SpectralAxisUnit" utype="ssa:Char.SpectralAxis.unit" value=""/>
      <PARAM ID="TimeLocation" datatype="double" name="OUTPUT:TimeLocation" ucd="time.epoch" unit="d" utype="ssa:Char.TimeAxis.Coverage.Location.Value" value="0">
       <DESCRIPTION>
        Estimated UT day of observations
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="SpatialLocation" arraysize="2" datatype="double" name="OUTPUT:SpatialLocation" ucd="pos.eq" unit="deg" utype="ssa:Char.SpatialAxis.Coverage.Location.Value" value="0">
       <DESCRIPTION>
        Spatial Position
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="SpectralLocation" datatype="double" name="OUTPUT:SpectralLocation" ucd="instr.bandpass" unit="AA" utype="ssa:Char.Spectral.Coverage.Location.Value" value="0">
       <DESCRIPTION>
        Midpoint of the spectral coverage in Angstroms
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="SpectralBoundsExtent" datatype="double" name="OUTPUT:SpectralBoundsExtent" ucd="instr.bandwidth" unit="AA" utype="ssa:Char.Spectral.Coverage.Bounds.Extent" value="0">
       <DESCRIPTION>
        Width of spectrum in Angstroms
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="SpectralBoundsStart" datatype="double" name="OUTPUT:SpectralBoundsStart" ucd="em.wl;stat.min" unit="AA" utype="ssa:Char.Spectral.Coverage.Bounds.Start" value="0">
       <DESCRIPTION>
        Start of spectral coordinate in Angstroms
       </DESCRIPTION>
      </PARAM>
      <PARAM ID="SpectralBoundsStop" datatype="double" name="OUTPUT:SpectralBoundsStop" ucd="em.wl;stat.max" unit="AA" utype="ssa:Char.Spectral.Coverage.Bounds.Stop" value="0">
       <DESCRIPTION>
        End of spectral coordinate in Angstroms
       </DESCRIPTION>
      </PARAM>
      <INFO ID="QUERY_STATUS" name="QUERY_STATUS" value="OK"/>
      <INFO ID="SERVICE_PROTOCOL" name="SERVICE_PROTOCOL" value="1.1"/>
     </RESOURCE>
    </VOTABLE>


----